<a href="https://colab.research.google.com/github/Penguinbeanie/Capstone-Project/blob/dev_branch/Capstone_Project_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Model

In [1]:
import google.generativeai as genai
from IPython.display import display, Markdown
from google.colab import userdata
import json

# retrieving the key stored in Colab
key = userdata.get('GOOGLE_API_KEY')

# configure the key for calling GenAI model
genai.configure(api_key=key)

# load model
model = genai.GenerativeModel("gemini-1.5-flash")

# Sorting Original Questionnaire Questions by Question Type

In [ ]:
import json
import requests
from google.colab import files

input_files = [
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire1.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire2.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire3.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire4.json",
    "https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/original_files/questionnaire5.json"
]

# Dictionary to hold questions grouped by type
question_types = {}

# Iterate through each file
for file_url in input_files:
    response = requests.get(file_url)
    data = response.json()
    for entry in data:
        q_type = entry['type']
        if q_type not in question_types:
            question_types[q_type] = []
        question_types[q_type].append(entry)

# Save each question type to separate JSON files
output_files = []
for q_type, questions in question_types.items():
    filename = f"/content/{q_type}.json"
    with open(filename, 'w') as f:
        json.dump(questions, f, indent=4)
    output_files.append(filename)

print("Files have been created.")


Files have been created.


# JSON To String For Context

In [ ]:
# Load files

import json
import requests

file_path_date = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/DATE.json'
file_path_multi = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/MULTI_SELECT.json'
file_path_number = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/NUMBER.json'
file_path_single = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/SINGLE_SELECT.json'
file_path_text = 'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/original_questionnaire/type_seperated/TEXT.json'

# Function to load JSON from URL
def load_json_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    return response.json()

# Load JSON data from URLs
try:
    json_data_date = load_json_from_url(file_path_date)
    json_data_multi = load_json_from_url(file_path_multi)
    json_data_number = load_json_from_url(file_path_number)
    json_data_single = load_json_from_url(file_path_single)
    json_data_text = load_json_from_url(file_path_text)

    # Convert JSON data to strings (if needed)
    json_string_date = json.dumps(json_data_date, indent=4)
    json_string_multi = json.dumps(json_data_multi, indent=4)
    json_string_number = json.dumps(json_data_number, indent=4)
    json_string_single = json.dumps(json_data_single, indent=4)
    json_string_text = json.dumps(json_data_text, indent=4)

except requests.exceptions.RequestException as e:
    print(f"Error loading JSON files: {e}")


# Running the Model

In [ ]:
# prompt

multiVar = 10
singleVar = 10
numberVar = 50
textVar = 50
dateVar = 50

prompt_multi = f"""
                Using the context below as a template, create {multiVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "MULTI-SELECT". The "options" array should contain multiple unique objects,
                each with the "option" key and a meaningful value representing an available choice. No IDs should be included.
                Generate multiple unique examples of questions that make sense in context for the type "MULTI-SELECT".
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Diverse.

                Context:
                {json_string_multi}
                """

prompt_single = f"""
                Using the context below as a template, create {singleVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "SINGLE-SELECT". The "options" array should contain multiple unique objects,
                each with the "option" key and a meaningful value representing an available choice. No IDs should be included.
                Generate multiple unique examples of questions that make sense in context for the type "SINGLE-SELECT".
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Communications.

                Context:
                {json_string_single}
                """

prompt_text = f"""
                Using the context below as a template, create {textVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "TEXT", the "options" array always contains exactly one object with
                the "option" set to "Text", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "TEXT". They should be open ended. Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Communications. (General topic in the field of communications, not directly the word communications)

                Context:
                {json_string_text}
                """

prompt_number = f"""
                Using the context below as a template, create {numberVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "NUMBER", the "options" array always contains exactly one object with
                the "option" set to the category the question best, and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "NUMBER". Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Convention/Fair.

                Context:
                {json_string_number}
                """

prompt_date = f"""
                Using the context below as a template, create {dateVar} more JSON objects. The output should include only the keys "type",
                "question", and "options". Ensure the "type" is always "DATE", the "options" array always contains exactly one object with
                the "option" set to "Date", and no IDs are included. Generate multiple unique examples of questions that make sense in context
                for the type "DATE". Make sure the question specifically ask for a date. Add "Provide a date." at the end of each question.
                Output only valid JSON, with proper line spacing and indentation, without any additional formatting or code block delimiters.

                Topic: Questionnnaire for evaluation. Healthcare.

                Context:
                {json_string_date}
                """


# responses
responses = {
    #"prompt_multi": "Questionnaire_Multi_Artificial.json",
    #"prompt_single": "Questionnaire_Single_Artificial.json",
    #"prompt_date": "Questionnaire_Date_Artificial.json",
    #"prompt_number": "Questionnaire_Number_Artificial.json",
    "prompt_text": "Questionnaire_Text_Artificial.json"
}

for response_name, response_file in responses.items():
    prompt = globals()[response_name]  # Get the prompt string
    response = model.generate_content(prompt)  # Get model's response

    try:
        response_data = json.loads(response.text)  # Parse the response text as JSON
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for {response_name}: {e}")
        continue

    # Write the parsed JSON to a file
    with open(response_file, "w") as json_file:
        json.dump(response_data, json_file, indent=4)

    print(f"JSON file '{response_file}' has been created")

JSON file 'Questionnaire_Text_Artificial.json' has been created


#JSON to Dataframe

###MULTI

In [2]:
import json
import pandas as pd
import requests

#Load the JSON data
questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_HeavyIndustry.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_Sales.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/MULTI/Questionnaire_Multi_Artificial_SoftwareDev.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data
    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_MULTI = pd.concat(dfs, ignore_index=True)

df_for_csv = df_MULTI.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('MULTI_combined.csv', index=False)

df_MULTI

,type,question,options
0,MULTI_SELECT,Which social media platforms do you use for ma...,"[Facebook, Instagram, Twitter, LinkedIn, TikTo..."
1,MULTI-SELECT,What types of marketing content do you find mo...,"[Blog posts, Videos, Infographics, Case studie..."
2,MULTI-SELECT,What are your primary goals for this marketing...,"[Increase brand awareness, Generate leads, Dri..."
3,MULTI-SELECT,What are your preferred methods of customer co...,"[Email, Phone, Chat, Social media, In-person m..."
4,MULTI-SELECT,What challenges are you facing in your current...,"[Lack of budget, Lack of time, Lack of resourc..."
...,...,...,...
186,MULTI-SELECT,What types of software development tools have ...,"[Integrated Development Environments (IDEs), S..."
187,MULTI-SELECT,What types of communication protocols are you ...,"[HTTP, HTTPS, TCP/IP, UDP, REST, GraphQL]"
188,MULTI-SELECT,What types of design patterns have you impleme...,"[Creational patterns, Structural patterns, Beh..."
189,MULTI-SELECT,What are your preferred methods for maintainin...,"[Code reviews, Static analysis, Unit testing, ..."


###SINGLE

In [11]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_ArtIndustry.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_Communications.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/main/artificial_questionnaire/SINGLE/Questionnaire_Single_Artificial_SoftwareDev.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_SINGLE = pd.concat(dfs, ignore_index=True)

df_for_csv = df_SINGLE.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('SINGLE_combined.csv', index=False)

df_SINGLE


,type,question,options
0,SINGLE-SELECT,What is your preferred medium for artistic exp...,"[Painting, Sculpture, Digital Art, Photography..."
1,SINGLE-SELECT,What style of art are you most drawn to?,"[Abstract, Realism, Surrealism, Impressionism,..."
2,SINGLE-SELECT,What is your primary role in the art and desig...,"[Artist, Designer, Art Director, Graphic Desig..."
3,SINGLE-SELECT,What software do you primarily use for your work?,"[Adobe Photoshop, Adobe Illustrator, Adobe InD..."
4,SINGLE-SELECT,What is your preferred color palette?,"[Warm Colors, Cool Colors, Monochromatic, Anal..."
...,...,...,...
145,SINGLE-SELECT,What is your experience with data streaming te...,"[Beginner, Intermediate, Advanced]"
146,SINGLE-SELECT,What is your experience with big data technolo...,"[Beginner, Intermediate, Advanced]"
147,SINGLE-SELECT,What is your experience with artificial intell...,"[Beginner, Intermediate, Advanced]"
148,SINGLE-SELECT,What is your experience with blockchain techno...,"[Beginner, Intermediate, Advanced]"


### NUMBER

In [24]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_BusinessAndJob.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_HeavyIndustries.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_Diverse.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/NUMBER/Questionnaire_Number_Artificial_Customer.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_NUMBER = pd.concat(dfs, ignore_index=True)

df_for_csv = df_NUMBER.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('NUMBER_combined.csv', index=False)

df_NUMBER

,type,question,options
0,NUMBER,What is your employee ID number?,[employee ID]
1,NUMBER,What is the project budget in dollars?,[budget]
2,NUMBER,How many employees are in the department?,[employee count]
3,NUMBER,What is the annual revenue target?,[revenue]
4,NUMBER,What is the client account number?,[account number]
...,...,...,...
180,NUMBER,What is your pant size?,[pant size]
181,NUMBER,How many times have you traveled internationally?,[international travels]
182,NUMBER,How many credit cards do you have?,[credit card count]
183,NUMBER,What is your estimated net worth?,[net worth]


### DATE

In [32]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_Customer.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_HeavyIndustries.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_Technology.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/DATE/Questionnaire_Date_Artificial_Healthcare.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_DATE = pd.concat(dfs, ignore_index=True)

df_for_csv = df_DATE.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('DATE_combined.csv', index=False)

df_DATE

,type,question,options
0,DATE,What is your date of birth? Provide a date.,[Date]
1,DATE,When did you purchase our product? Provide a d...,[Date]
2,DATE,What date did the issue begin? Provide a date.,[Date]
3,DATE,When is your next appointment? Provide a date.,[Date]
4,DATE,What is the delivery date? Provide a date.,[Date]
...,...,...,...
183,DATE,When did you last consume alcohol? Provide a d...,[Date]
184,DATE,When was your last physical therapy appointmen...,[Date]
185,DATE,When did you last have a significant injury? P...,[Date]
186,DATE,When did you start experiencing shortness of b...,[Date]


### TEXT

In [33]:
import json
import pandas as pd
import requests

#Load the JSON data

questions = [
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_Communications.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_HeavyIndustries.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_CustomerSupport.json',
    'https://raw.githubusercontent.com/Penguinbeanie/Capstone-Project/refs/heads/dev_branch/artificial_questionnaire/TEXT/Questionnaire_Text_Artificial_Convention.json'
]

dfs = []

for question in questions:
    response = requests.get(question)
    data = response.json()

    #Create a DataFrame from the JSON data

    df_temp = pd.DataFrame(data)
    df_temp['options'] = df_temp['options'].apply(lambda x: [item['option'] for item in x])
    dfs.append(df_temp)

df_TEXT = pd.concat(dfs, ignore_index=True)

df_for_csv = df_TEXT.copy()
df_for_csv['options'] = df_for_csv['options'].apply(lambda x: '; '.join(x))
df_for_csv.to_csv('TEXT_combined.csv', index=False)

df_TEXT

,type,question,options
0,TEXT,Describe your experience.,[Text]
1,TEXT,What were the key takeaways?,[Text]
2,TEXT,Explain your approach.,[Text]
3,TEXT,Summarize the main points.,[Text]
4,TEXT,What challenges did you encounter?,[Text]
...,...,...,...
173,TEXT,Describe any areas where the convention fell s...,[Text]
174,TEXT,What aspects of the convention fostered a sens...,[Text]
175,TEXT,How effectively did the convention balance com...,[Text]
176,TEXT,What were the strengths and weaknesses of the ...,[Text]


# Answer Generation

### MULTI

In [5]:
import time
from google.api_core import exceptions

df_MULTI_with_answers = df_MULTI.iloc[100:190].copy()

# Generate a single compact prompt for all variations
def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {', '.join(row['options'])}

            Provide 5 different responses to this multiple choice question. Each response should select from the given options. Chose a different
            cobination of options (or single option) for each response. Each answer (V, C, Q, E, A) is to be given by a seperate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain the options corresponding to the response.
            Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [options corresponding to the response V]
            C: [concise response]
            C_Sel: [options corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [options corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [options corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [options corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

# Parse the response text into separate variations
def parse_responses(text):
    answers = {}
    lines = text.split('\n')

    # Extract each response type
    for key, prefix in prefixes.items():
        try:
            # Find the line starting with this prefix
            response_line = next((line for line in lines if line.strip().startswith(prefix)), '')
            # Remove the prefix and trim
            response = response_line.replace(prefix, '', 1).strip()
            answers[f"answer_{key}"] = response if response else f"Error: No {key} response found"
        except Exception as e:
            answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

# Process a single row with retries
def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

# Process the dataframe
answer_columns = df_MULTI_with_answers.apply(process_row, axis=1)

# Add columns to dataframe
for key in prefixes:
    df_MULTI_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

# Save to CSV
df_for_csv_MULTI_answer = df_MULTI_with_answers.copy()
df_for_csv_MULTI_answer['options'] = df_for_csv_MULTI_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_MULTI_answer.to_csv('MULTI_answer_combined.csv', index=False, sep='>')

df_MULTI_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
100,MULTI-SELECT,What are your current marketing and sales tech...,"[CRM (Salesforce, HubSpot, etc.), Marketing au...",We utilize a comprehensive suite of marketing ...,"CRM (Salesforce, HubSpot, etc.), Marketing aut...","Salesforce, Google Analytics, and a social med...","CRM (Salesforce, HubSpot, etc.), Analytics pla...","We're all over the place, man! Got Salesforce...","CRM (Salesforce, HubSpot, etc.), Email marketi...",Our sales team relies heavily on Salesforce fo...,"CRM (Salesforce, HubSpot, etc.), Marketing aut...","We use Salesforce, that's the main one. And o...","CRM (Salesforce, HubSpot, etc.), Analytics pla..."
101,MULTI-SELECT,What are your sales team's key performance ind...,"[Revenue generated, Number of deals closed, Av...","Our sales team's KPIs are multifaceted, focusi...","Revenue generated, Number of deals closed, Ave...","Revenue generated, number of deals closed.","Revenue generated, Number of deals closed",We totally look at how much money we're bringi...,"Revenue generated, Number of deals closed, Sal...","To assess our sales team's effectiveness, we p...","Revenue generated, Number of deals closed, Ave...",The main ones are revenue and the number of de...,"Revenue generated, Number of deals closed, Cus..."
102,MULTI-SELECT,What are your company's key differentiators in...,"[Superior product quality, Lower prices, Excel...",Our company stands out due to its unwavering c...,"Superior product quality, Excellent customer s...",Superior product quality and lower prices.,"Superior product quality, Lower prices","We totally rock at customer service, and our p...","Excellent customer service, Lower prices",Our key differentiators are a combination of i...,"Innovative technology, Superior product quality","It's a combination of factors, really. Our st...","Strong brand reputation, Superior product quality"
103,MULTI-SELECT,What are your top three priorities for improvi...,"[Improving lead generation, Increasing convers...",My top priorities are boosting lead generation...,"Improving lead generation, Improving sales tea...","Lead generation, conversion rates, and sales c...","Improving lead generation, Increasing conversi...","Gotta get more leads, you know? Then, gotta c...","Improving lead generation, Increasing conversi...","To improve our sales process, we need to focus...","Improving lead generation, Increasing conversi...","Frankly, we need better leads. Then we need t...","Improving lead generation, Increasing conversi..."
104,MULTI-SELECT,What types of content do you find most effecti...,"[Blog posts, Case studies, White papers, Webin...","For lead generation, I find a combination of h...","Case studies, Webinars, Ebooks, Blog posts",Webinars are most effective.,Webinars,"Gotta say, ebooks and infographics are killer...","Ebooks, Infographics",I've found that a multi-faceted approach is mo...,"Blog posts, Ebooks, White papers, Case studies","It depends on your target audience, but genera...","Case studies, White papers"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,MULTI-SELECT,What are your preferred methods for code refac...,"[Extract Method, Rename Method, Move Method, E...",My preferred refactoring methods often involve...,"Extract Method, Rename Method, Move Method",Extract Method is my go-to.,Extract Method,I usually start by pulling out chunks of code ...,"Extract Method, Rename Method",For improving code clarity and maintainability...,"Extract Method, Rename Method, Extract Class","It depends on the situation, but often it's ju...","Rename Method, Extract Method, Move Method"
186,MULTI-SELECT,What types of software development tools have ...,"[Integrated Development Environments (IDEs), S...",I've extensively used Integrated Developm

In [22]:
df_0_100 = pd.read_csv('MULTI_answer_combined_0-100.csv', sep = '>')
df_101_189 = pd.read_csv('MULTI_answer_combined_101-189.csv', sep = '>')

df_MULTI_with_answers_total = pd.concat([df_0_100, df_101_189], ignore_index=True)
df_MULTI_with_answers_total.to_csv('MULTI_answer_combined_total.csv', index=False, sep='>')


In [23]:
import pandas as pd

# Step 1: Melt the dataframe to combine all answer columns into one column
df_long = df_MULTI_with_answers_total.melt(
    id_vars=["type", "question", "options"],
    var_name="answer_type",
    value_name="answer_value"
)

# Step 2: Create a new column for the answer type (e.g., verbose, concise, etc.)
df_long["answer_category"] = df_long["answer_type"].str.extract(r"answer_(\w+)")

# Step 3: Separate rows into answers and their labels
answers = df_long[~df_long["answer_type"].str.endswith("_Gemini_label")].copy()
labels = df_long[df_long["answer_type"].str.endswith("_Gemini_label")].copy()

# Step 4: Add a matching key column to facilitate merging
answers["label_type"] = answers["answer_type"] + "_Gemini_label"

# Step 5: Merge answers with their corresponding labels
df_reshaped = pd.merge(
    answers,
    labels,
    left_on=["type", "question", "options", "label_type"],
    right_on=["type", "question", "options", "answer_type"],
    suffixes=("", "_label")
)

# Step 6: Rename columns and keep the desired ones
df_reshaped = df_reshaped.rename(
    columns={"answer_value": "answer", "answer_value_label": "answer_label_GEMINI"}
)[["type", "question", "options", "answer", "answer_label_GEMINI", "answer_category"]]

# Step 7: Sort the data if needed (optional)
df_reshaped_MULTI = df_reshaped.sort_values(by=["type", "question"]).reset_index(drop=True)

# Display the final dataframe
df_reshaped_MULTI.to_csv('MULTI_answer_combined_total_reshaped.csv', index=False, sep='>')
df_reshaped_MULTI


,type,question,options,answer,answer_label_GEMINI,answer_category
0,MULTI-SELECT,What algorithms are you familiar with?,Sorting algorithms; Searching algorithms; Grap...,"I'm familiar with a wide range of algorithms, ...","[Sorting algorithms, Searching algorithms, Gra...",verbose
1,MULTI-SELECT,What algorithms are you familiar with?,Sorting algorithms; Searching algorithms; Grap...,"Sorting, searching, and graph algorithms.","[Sorting algorithms, Searching algorithms, Gra...",concise
2,MULTI-SELECT,What algorithms are you familiar with?,Sorting algorithms; Searching algorithms; Grap...,"I know a bunch of those, like the sorting and ...","[Sorting algorithms, Searching algorithms, Gra...",colloquial
3,MULTI-SELECT,What algorithms are you familiar with?,Sorting algorithms; Searching algorithms; Grap...,My algorithm knowledge encompasses several cat...,"[Sorting algorithms, Searching algorithms, Dyn...",explanatory
4,MULTI-SELECT,What algorithms are you familiar with?,Sorting algorithms; Searching algorithms; Grap...,"I know about sorting, searching, and divide an...","[Sorting algorithms, Searching algorithms, Div...",annoyed
...,...,...,...,...,...,...
945,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,We primarily utilize Facebook and Instagram fo...,"Facebook, Instagram, LinkedIn",verbose
946,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,"Facebook, Instagram, and TikTok.","Facebook, Instagram, TikTok",concise
947,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,"We're all over Facebook and Insta, mostly. Yo...","Facebook, Instagram, YouTube",colloquial
948,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,Our marketing strategy uses a multi-platform a...,"Facebook, Instagram, YouTube, LinkedIn",explanatory


### SINGLE

In [13]:
import time
from google.api_core import exceptions

df_SINGLE_with_answers = df_SINGLE.iloc[101:150].copy()

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {', '.join(row['options'])}
            Provide 5 different responses to this single choice question. Each response should select from the given options. Chose a different
            option for each response (if enough options exist for all 5 responses, otherwise repeat options after having already used all of them).
            Each answer (V, C, Q, E, A) is to be given by a seperate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain the option corresponding to the response. Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [option corresponding to the response V]
            C: [concise response]
            C_Sel: [option corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [option corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [option corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [option corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_SINGLE_with_answers.apply(process_row, axis=1)

# Add columns to dataframe
for key in prefixes:
    df_SINGLE_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_SINGLE_answer = df_SINGLE_with_answers.copy()
df_for_csv_SINGLE_answer['options'] = df_for_csv_SINGLE_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_SINGLE_answer.to_csv('SINGLE_answer_combined.csv', index=False, sep='>')

df_SINGLE_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
101,SINGLE-SELECT,What is your primary source of leads?,"[Inbound Marketing, Outbound Sales, Referrals,...",Our primary source of leads is inbound market...,Inbound Marketing,Referrals.,Referrals,"Partnerships, dude. That's where it's at.",Partnerships,Our primary lead generation method is outboun...,Outbound Sales,"It's mainly inbound marketing, if you must ...",Inbound Marketing
102,SINGLE-SELECT,What is your company's mission?,"[Profit Maximization, Social Impact, Environme...",Our company strives to create a positive impa...,Social Impact,Customer satisfaction is key.,Customer Satisfaction,"Dude, it's all about making bank, you know? ...",Profit Maximization,Our primary goal is to minimize our environm...,Environmental Sustainability,The question should be obvious. We aim to k...,Customer Satisfaction
103,SINGLE-SELECT,What is your company culture?,"[Formal, Informal, Hybrid]",We maintain a professional atmosphere with cl...,Formal,Formal.,Formal,"It's pretty relaxed, you know? Not all stuff...",Informal,Our company culture is a blend of formal and ...,Hybrid,"It depends on what you mean by ""company cultu...",Hybrid
104,SINGLE-SELECT,What is your preferred integration method?,"[API, CSV Import/Export, Other]","For seamless and real-time data exchange, an...",API,API.,API,"Gotta go with the API, man. That's the slic...",API,"CSV import/export is suitable for simpler, le...",CSV Import/Export,"Fine, if I *must* choose, I suppose CSV Impo...",CSV Import/Export
105,SINGLE-SELECT,What is your current level of digital transfor...,"[Beginner, Intermediate, Advanced]",We're just starting to explore digital tools ...,Beginner,Beginner.,Beginner,"We're total newbies at this digital stuff, s...",Beginner,Our digital transformation is in its initial ...,Intermediate,"We're pretty far along in some areas, but st...",Advanced
106,SINGLE-SELECT,What are your top three priorities for this year?,"[Revenue Growth, Cost Reduction, Customer Acqu...",Our top priority this year is to significant...,Revenue Growth,Customer Acquisition is key.,Customer Acquisition,"Gotta get more peeps using our stuff, so Cust...",Customer Acquisition,Cost Reduction is paramount this year. By s...,Cost Reduction,"If I had to pick just one, it's Product Deve...",Product Development
107,SINGLE-SELECT,What is your biggest challenge in your current...,"[Lack of Resources, Lack of Time, Lack of Supp...",My biggest challenge is definitely the lack o...,Lack of Resources,Lack of time.,Lack of Time,Biggest hurdle? Totally running out of time...,Lack of Time,The insufficient support from other departme...,Lack of Support,"It's frustrating, the lack of resources is r...",Lack of Resources
108,SINGLE-SELECT,What is your team size?,"[1-5, 6-10, 11-20, 20+]","We're a fairly small team, currently operatin...",1-5,1-5.,1-5,"Just me and a couple of others, so 1-5 I guess.",1-5,Our team consists of eleven to twenty individ...,11-20,It's 20+. Why do you need to know?,20+
109,SINGLE-SELECT,What is your company's main focus?,"[B2B, B2C, Both]",Our company primarily focuses on business-to-...,B2B,B2B.,B2B,"We're all about the biz-to-biz action, dude. ...",B2B,Our main focus is the business-to-consumer m...,B2C,It's B2B. Is that clear enough?,B2B
110,SINGLE-SELECT,How would you rate your experience with our cu...,"[Excellent, Good, Fair, Poor]",My experience with your customer support was ...,Excellent,Good.,Good,"It was alright, I guess. Coulda been better,...",Fair,The support I received was fair. While the r...,Fair,The service was quite poor; it took far too ...,Poor


In [18]:
df_0_100 = pd.read_csv('SINGLE_answer_combined_0-100.csv', sep = '>')
df_101_189 = pd.read_csv('SINGLE_answer_combined_101-149.csv', sep = '>')

df_SINGLE_with_answers_total = pd.concat([df_0_100, df_101_189], ignore_index=True)
df_SINGLE_with_answers_total.to_csv('SINGLE_answer_combined_total.csv', index=False, sep='>')

In [20]:
import pandas as pd

# Step 1: Melt the dataframe to combine all answer columns into one column
df_long = df_SINGLE_with_answers_total.melt(
    id_vars=["type", "question", "options"],
    var_name="answer_type",
    value_name="answer_value"
)

# Step 2: Create a new column for the answer type (e.g., verbose, concise, etc.)
df_long["answer_category"] = df_long["answer_type"].str.extract(r"answer_(\w+)")

# Step 3: Separate rows into answers and their labels
answers = df_long[~df_long["answer_type"].str.endswith("_Gemini_label")].copy()
labels = df_long[df_long["answer_type"].str.endswith("_Gemini_label")].copy()

# Step 4: Add a matching key column to facilitate merging
answers["label_type"] = answers["answer_type"] + "_Gemini_label"

# Step 5: Merge answers with their corresponding labels
df_reshaped = pd.merge(
    answers,
    labels,
    left_on=["type", "question", "options", "label_type"],
    right_on=["type", "question", "options", "answer_type"],
    suffixes=("", "_label")
)

# Step 6: Rename columns and keep the desired ones
df_reshaped = df_reshaped.rename(
    columns={"answer_value": "answer", "answer_value_label": "answer_label_GEMINI"}
)[["type", "question", "options", "answer", "answer_label_GEMINI", "answer_category"]]

# Step 7: Sort the data if needed (optional)
df_reshaped_SINGLE = df_reshaped.sort_values(by=["type", "question"]).reset_index(drop=True)

# Display the final dataframe
df_reshaped_SINGLE.to_csv('SINGLE_answer_combined_total_reshaped.csv', index=False, sep='>')
df_reshaped_SINGLE

,type,question,options,answer,answer_label_GEMINI,answer_category
0,SINGLE-SELECT,Are you currently using a similar product?,Yes; No,"Yes, I'm currently utilizing a similar produc...",Yes,verbose
1,SINGLE-SELECT,Are you currently using a similar product?,Yes; No,No.,No,concise
2,SINGLE-SELECT,Are you currently using a similar product?,Yes; No,"Nope, nothin' like that here.",No,colloquial
3,SINGLE-SELECT,Are you currently using a similar product?,Yes; No,"No, I am not currently using a similar produc...",No,explanatory
4,SINGLE-SELECT,Are you currently using a similar product?,Yes; No,"No, I already have a perfectly adequate syst...",No,annoyed
...,...,...,...,...,...,...
740,SINGLE_SELECT,What is your preferred client type?,Small Businesses; Large Corporations; Non-prof...,I find working with small businesses the most...,Small Businesses,verbose
741,SINGLE_SELECT,What is your preferred client type?,Small Businesses; Large Corporations; Non-prof...,Large corporations.,Large Corporations,concise
742,SINGLE_SELECT,What is your preferred client type?,Small Businesses; Large Corporations; Non-prof...,"Gotta go with non-profits, man. Doing good ...",Non-profits,colloquial
743,SINGLE_SELECT,What is your preferred client type?,Small Businesses; Large Corporations; Non-prof...,Government agencies offer a unique set of ch...,Government Agencies,explanatory


### NUMBER

In [25]:
import time
from google.api_core import exceptions

df_NUMBER_with_answers = df_NUMBER.copy()

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {row['options']}
            Provide 5 different responses to this single choice question, which asks for a number. The question provides a single option defining the type of number to be chosen.
            Each response should adhere to the given type but provide a unique answer. Some answers should be written as numbers, some such that the numbers are spelled out.
            Each answer (V, C, Q, E, A) is to be given by a separate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain just the number (with correct units) corresponding to the response. Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [number corresponding to the response V]
            C: [concise response]
            C_Sel: [number corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [number corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [number corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [number corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_NUMBER_with_answers.apply(process_row, axis=1)

for key in prefixes:
    df_NUMBER_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_NUMBER_answer = df_NUMBER_with_answers.copy()
df_for_csv_NUMBER_answer['options'] = df_for_csv_NUMBER_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_NUMBER_answer.to_csv('NUMBER_answer_combined.csv', index=False, sep='>')

df_NUMBER_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,NUMBER,What is your employee ID number?,[employee ID],My employee ID number is assigned by the comp...,1234567,8901234,8901234,"It's, like, 9876543, man.",9876543,My employee ID is a seven-digit number used t...,2468013,"My employee ID is five million, four hundred...",5432100
1,NUMBER,What is the project budget in dollars?,[budget],"The project budget is substantial, totaling o...","$1,200,000","$500,000","$500,000","The budget? It's, like, a cool three millio...","$3,000,000","The project budget is set at $750,000. This a...","$750,000",The budget is twenty-five thousand dollars. ...,"$25,000"
2,NUMBER,How many employees are in the department?,[employee count],There are twenty-seven employees currently wo...,27 employees,15,15 employees,"There's like, forty-two people in the depart...",42 employees,The employee count for this department is pre...,31 employees,The department has eighty-five employees. I...,85 employees
3,NUMBER,What is the annual revenue target?,[revenue],Our projected annual revenue target is a sign...,"$5,000,000",Five million dollars.,"$5,000,000",We're shootin' for five million bucks this y...,"$5,000,000",The annual revenue target has been carefully ...,"$10,000,000",The target is twelve million dollars. Is th...,"$12,000,000"
4,NUMBER,What is the client account number?,[account number],"My client account number, as requested, is on...",1234567,87654321,87654321,The account number? It's 9876543. Is that ...,9876543,The client account number you requested is 11...,11223344,The account number is 5551212. I've already...,5551212
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,NUMBER,What is your pant size?,[pant size],My pants size is usually a thirty-four. It c...,34,32,32,"I'd say about a thirty-six, maybe a little b...",36,My pant size is 30. This is based on recent ...,30,It's 28. Why do you need to know?,28
181,NUMBER,How many times have you traveled internationally?,[international travels],I've had the pleasure of traveling internatio...,5 international travels,Three.,3 international travels,"Like, maybe seven times? Lost count a bit.",7 international travels,I have undertaken international travel on two...,2 international travels,I've been abroad twelve times. It's really ...,12 international travels
182,NUMBER,How many credit cards do you have?,[credit card count],I currently have three credit cards. Two are...,3 credit cards,Two.,2 credit cards,"Just a couple, you know?",2 credit cards,I maintain two credit cards for responsible f...,2 credit cards,One. It's really none of your business.,1 credit card
183,NUMBER,What is your estimated net worth?,[net worth],"My estimated net worth is quite low, as I am...",$0,Zero.,$0,"Dude, I'm a computer program. I ain't got n...",$0,"As a large language model, I don't own any p...",$0,Seriously? My net worth is zero. I'm an AI.,$0


In [30]:
import pandas as pd

# Convert options to strings before melting
df_NUMBER_with_answers['options'] = df_NUMBER_with_answers['options'].astype(str)

# Step 1: Melt the dataframe to combine all answer columns into one column
df_long = df_NUMBER_with_answers.melt(
    id_vars=["type", "question", "options"],
    var_name="answer_type",
    value_name="answer_value"
)


# Step 2: Create a new column for the answer type (e.g., verbose, concise, etc.)
df_long["answer_category"] = df_long["answer_type"].str.extract(r"answer_(\w+)")

# Step 3: Separate rows into answers and their labels
answers = df_long[~df_long["answer_type"].str.endswith("_Gemini_label")].copy()
labels = df_long[df_long["answer_type"].str.endswith("_Gemini_label")].copy()

# Step 4: Add a matching key column to facilitate merging
answers["label_type"] = answers["answer_type"] + "_Gemini_label"

# Step 5: Merge answers with their corresponding labels
df_reshaped = pd.merge(
    answers,
    labels,
    left_on=["type", "question", "options", "label_type"],
    right_on=["type", "question", "options", "answer_type"],
    suffixes=("", "_label")
)

# Step 6: Rename columns and keep the desired ones
df_reshaped = df_reshaped.rename(
    columns={"answer_value": "answer", "answer_value_label": "answer_label_GEMINI"}
)[["type", "question", "options", "answer", "answer_label_GEMINI", "answer_category"]]

# Step 7: Sort the data if needed (optional)
df_reshaped_NUMBER = df_reshaped.sort_values(by=["type", "question"]).reset_index(drop=True)

# Display the final dataframe
df_reshaped_NUMBER.to_csv('NUMBER_answer_combined_total_reshaped.csv', index=False, sep='>')
df_reshaped_NUMBER

        type                                           question  \
0     NUMBER                   What is your employee ID number?   
1     NUMBER             What is the project budget in dollars?   
2     NUMBER          How many employees are in the department?   
3     NUMBER                 What is the annual revenue target?   
4     NUMBER                 What is the client account number?   
...      ...                                                ...   
1845  NUMBER                            What is your pant size?   
1846  NUMBER  How many times have you traveled internationally?   
1847  NUMBER                 How many credit cards do you have?   
1848  NUMBER                  What is your estimated net worth?   
1849  NUMBER           How many years of education do you have?   

                        options                  answer_type  \
0               ['employee ID']               answer_verbose   
1                    ['budget']               answer_verbose   
2  

,type,question,options,answer,answer_label_GEMINI,answer_category
0,NUMBER,How many attendees are expected at the confere...,['attendee count'],We anticipate a significant turnout for the c...,500 attendees,verbose
1,NUMBER,How many attendees are expected at the confere...,['attendee count'],1200,1200 attendees,concise
2,NUMBER,How many attendees are expected at the confere...,['attendee count'],"Like, a thousand people, maybe a bit more, I...",1000+ attendees,colloquial
3,NUMBER,How many attendees are expected at the confere...,['attendee count'],The projected attendee count is based on a co...,750 attendees,explanatory
4,NUMBER,How many attendees are expected at the confere...,['attendee count'],The expected attendance? It's two thousand. ...,2000 attendees,annoyed
...,...,...,...,...,...,...
930,NUMBER,What year was the company founded?,['year'],The company's founding year is documented in ...,1987,verbose
931,NUMBER,What year was the company founded?,['year'],1992,1992,concise
932,NUMBER,What year was the company founded?,['year'],"It was, like, way back in two thousand and o...",2001,colloquial
933,NUMBER,What year was the company founded?,['year'],The company was established in 2015. This da...,2015,explanatory


### DATE

In [34]:
import time
from google.api_core import exceptions

df_DATE_with_answers = df_DATE.copy()

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {row['options']}
            Provide 5 different responses to this single choice question, which asks for a date. The question provides one option defining the type to be chosen.
            Each response should adhere to the given type but provide a unique answer. Some answers should be written as numbers, some such that the numbers are spelled out. Use different date formats.
            Assume question takers are from Europe.
            Each answer (V, C, Q, E, A) is to be given by a separate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain just the date corresponding to the response. Use this date format: dd-mm-yyyy and nothing else for the label.
            Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [date corresponding to the response V]
            C: [concise response]
            C_Sel: [date corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [date corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [date corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [date corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_DATE_with_answers.apply(process_row, axis=1)

for key in prefixes:
    df_DATE_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_DATE_answer = df_DATE_with_answers.copy()
df_for_csv_DATE_answer['options'] = df_for_csv_DATE_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_DATE_answer.to_csv('DATE_answer_combined.csv', index=False, sep='>')

df_DATE_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,DATE,What is your date of birth? Provide a date.,[Date],My birthday is on the twenty-fifth of Decembe...,25-12-1987,14/03/1992,14-03-1992,"It's the seventh of August, nineteen ninety-...",07-08-1995,My date of birth is 01/01/2000. This is the ...,01-01-2000,My birthday is 29-02-1988. Why do you need ...,29-02-1988
1,DATE,When did you purchase our product? Provide a d...,[Date],I purchased your product on the twenty-fifth ...,25-07-2022,12/03/2023,12-03-2023,"Got it sometime in April last year, mate. A...",10-04-2022,"My purchase date was October 18th, 2021. I'm...",18-10-2021,"It was the first of June, twenty-twenty-one....",01-06-2021
2,DATE,What date did the issue begin? Provide a date.,[Date],The problem started on the twenty-fifth of Oc...,25-10-2022,14/03/2023,14-03-2023,"It all kicked off on the first of May, last ...",01-05-2022,The issue commenced on 02 August 2021. This ...,02-08-2021,"The date was the twelfth of June, two thousa...",12-06-2021
3,DATE,When is your next appointment? Provide a date.,[Date],My next appointment is scheduled for the twen...,25-10-2024,14/03/2025,14-03-2025,"Next appt? See ya on the 2nd of June, next ...",02-06-2025,"My appointment is on the 18th of August, 2024...",18-08-2024,"It's on the seventh of January, twenty-twent...",07-01-2025
4,DATE,What is the delivery date? Provide a date.,[Date],The package will arrive on the twenty-fifth o...,25-10-2024,12/03/2025,12-03-2025,"It'll be here on the fifth of next month, ye...",05-11-2024,The delivery is scheduled for January fourtee...,14-01-2025,"The date is June twenty-second, twenty-twent...",22-06-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,DATE,When did you last consume alcohol? Provide a d...,[Date],I last enjoyed a glass of wine on the twenty-...,27-10-2022,14/03/2023,14-03-2023,"Ages ago, mate. Think it was sometime in Fe...",12-02-2023,My last alcoholic beverage was consumed on t...,31-12-2022,"It was June the fifth, two thousand and twen...",05-06-2023
184,DATE,When was your last physical therapy appointmen...,[Date],My last physical therapy appointment was on t...,27-03-2024,14/08/2023,14-08-2023,"Oh, it was ages ago, mate. Think it was som...",14-07-2024,My last physical therapy session was held on ...,10-10-2023,"It was December twelfth, two thousand and tw...",12-12-2022
185,DATE,When did you last have a significant injury? P...,[Date],I twisted my ankle rather badly playing footb...,27-10-2017,01/04/2022,01-04-2022,"Ages ago, man! Broke my arm, think it was ...",15-06-2015,I suffered a concussion during a cycling acci...,12-03-2021,It was in August two thousand and nineteen. ...,18-08-2019
186,DATE,When did you start experiencing shortness of b...,[Date],I first noticed I was short of breath around ...,05-12-2022,24/03/2023,24-03-2023,"Oh, it started sometime in September, I reck...",19-09-2023,"The shortness of breath began subtly, but it ...",27-08-2023,"It was around the eleventh of June, this year...",11-06-2024


In [35]:
import pandas as pd

# Convert options to strings before melting
df_DATE_with_answers['options'] = df_DATE_with_answers['options'].astype(str)

# Step 1: Melt the dataframe to combine all answer columns into one column
df_long = df_DATE_with_answers.melt(
    id_vars=["type", "question", "options"],
    var_name="answer_type",
    value_name="answer_value"
)


# Step 2: Create a new column for the answer type (e.g., verbose, concise, etc.)
df_long["answer_category"] = df_long["answer_type"].str.extract(r"answer_(\w+)")

# Step 3: Separate rows into answers and their labels
answers = df_long[~df_long["answer_type"].str.endswith("_Gemini_label")].copy()
labels = df_long[df_long["answer_type"].str.endswith("_Gemini_label")].copy()

# Step 4: Add a matching key column to facilitate merging
answers["label_type"] = answers["answer_type"] + "_Gemini_label"

# Step 5: Merge answers with their corresponding labels
df_reshaped = pd.merge(
    answers,
    labels,
    left_on=["type", "question", "options", "label_type"],
    right_on=["type", "question", "options", "answer_type"],
    suffixes=("", "_label")
)

# Step 6: Rename columns and keep the desired ones
df_reshaped = df_reshaped.rename(
    columns={"answer_value": "answer", "answer_value_label": "answer_label_GEMINI"}
)[["type", "question", "options", "answer", "answer_label_GEMINI", "answer_category"]]

# Step 7: Sort the data if needed (optional)
df_reshaped_DATE = df_reshaped.sort_values(by=["type", "question"]).reset_index(drop=True)

# Display the final dataframe
df_reshaped_DATE.to_csv('DATE_answer_combined_total_reshaped.csv', index=False, sep='>')
df_reshaped_DATE

,type,question,options,answer,answer_label_GEMINI,answer_category
0,DATE,On what date did the project officially commen...,['Date'],The project officially started on the twenty-...,23-03-2022,verbose
1,DATE,On what date did the project officially commen...,['Date'],01/01/2023,01-01-2023,concise
2,DATE,On what date did the project officially commen...,['Date'],"It kicked off on the tenth of June, twenty-t...",10-06-2021,colloquial
3,DATE,On what date did the project officially commen...,['Date'],The official commencement date of the project...,15-08-2024,explanatory
4,DATE,On what date did the project officially commen...,['Date'],The start date is clearly documented as 27 Se...,27-09-2020,annoyed
...,...,...,...,...,...,...
955,DATE,When will the product be ready for pickup? Pro...,['Date'],Your product will be ready for collection on ...,25-10-2024,verbose
956,DATE,When will the product be ready for pickup? Pro...,['Date'],12/03/2025,12-03-2025,concise
957,DATE,When will the product be ready for pickup? Pro...,['Date'],Ready for pick-up on the fifteenth of next m...,15-11-2024,colloquial
958,DATE,When will the product be ready for pickup? Pro...,['Date'],The product will be available for pickup on J...,18-01-2025,explanatory


### TEXT

In [36]:
import time
from google.api_core import exceptions

df_TEXT_with_answers = df_TEXT.copy()

def generate_minimal_prompt(row):
    return f"""Question: {row['question']}
            Options: {row['options']}
            Provide 5 different responses to this single choice question, which asks for a text. The question provides a single option defining the type of answer to be chosen.
            Each response should adhere to the given type but provide a unique answer. The questions are open ended, therfore your answer should replicate a human's answer to such a question.
            Each answer (V, C, Q, E, A) is to be given by a separate person.
            Each label (V_Sel, C_Sel, Q_Sel, E_Sel, A_Sel) should contain the shortest possible topic summery.
            Format exactly as follows:
            V: [verbose response, not exceeding 3 sentences]
            V_Sel: [summery corresponding to the response V]
            C: [concise response]
            C_Sel: [summery corresponding to the response C]
            Q: [colloquial response, no 'honestly']
            Q_Sel: [summery corresponding to the response Q]
            E: [explanatory response, not exceeding 3 sentences]
            E_Sel: [summery corresponding to the response E]
            A: [mildly annoyed response, no 'ugh']
            A_Sel: [summery corresponding to the response A]"""

# Define the prefixes for each response type
prefixes = {
    'verbose': 'V:',
    'verbose_Gemini_label': 'V_Sel: ',
    'concise': 'C:',
    'concise_Gemini_label': 'C_Sel: ',
    'colloquial': 'Q:',
    'colloquial_Gemini_label': 'Q_Sel: ',
    'explanatory': 'E:',
    'explanatory_Gemini_label': 'E_Sel: ',
    'annoyed': 'A:',
    'annoyed_Gemini_label': 'A_Sel: '
}

def parse_responses(text):
    answers = {}
    lines = text.split('\n')
    for key, prefix in prefixes.items():
        try:
            for line in lines:
                if line.startswith(prefix):
                    line = line.removeprefix(prefix)
                    answers[f"answer_{key}"] = line if line else f"Error: No {key} response found"
        except Exception as e:
                answers[f"answer_{key}"] = f"Error parsing {key} response: {str(e)}"

    return answers

def process_row(row):
    prompt = generate_minimal_prompt(row)
    for attempt in range(3):
        try:
            response = model.generate_content(prompt)
            answers = parse_responses(response.text)
            time.sleep(3)
            return answers
        except exceptions.TooManyRequests:
            if attempt < 2:
                time.sleep(2 ** attempt)
            else:
                return {f"answer_{key}": "Error: Rate limit exceeded."
                       for key in prefixes}
        except Exception as e:
            print(f"Generated text was:\n{response.text}\n")  # Print the response text
            print(f"Error: {str(e)}")
            return {f"answer_{key}": f"Error: {str(e)}"
                  for key in prefixes}

answer_columns = df_TEXT_with_answers.apply(process_row, axis=1)

for key in prefixes:
    df_TEXT_with_answers[f"answer_{key}"] = answer_columns.apply(lambda x: x[f"answer_{key}"])

df_for_csv_TEXT_answer = df_TEXT_with_answers.copy()
df_for_csv_TEXT_answer['options'] = df_for_csv_TEXT_answer['options'].apply(lambda x: '; '.join(x))
df_for_csv_TEXT_answer.to_csv('TEXT_answer_combined.csv', index=False, sep='>')

df_TEXT_with_answers

,type,question,options,answer_verbose,answer_verbose_Gemini_label,answer_concise,answer_concise_Gemini_label,answer_colloquial,answer_colloquial_Gemini_label,answer_explanatory,answer_explanatory_Gemini_label,answer_annoyed,answer_annoyed_Gemini_label
0,TEXT,Describe your experience.,[Text],My experience has been a rollercoaster of lea...,Personal growth journey,Varied and challenging.,Varied experiences,"It's been a wild ride, let me tell you! Ups...",Life's ups and downs,My experience encompasses a broad range of si...,Broad range of experiences shaping perspective,"It's been a mixed bag, to put it mildly. So...",Mixed experiences
1,TEXT,What were the key takeaways?,[Text],The key takeaways were the significant impact...,"Climate change, cooperation, renewable energy.","Climate change impacts, global cooperation ne...",Key environmental findings.,"So, like, the big things were how messed up ...","Climate change severity, global collaboration,...",The presentation highlighted three crucial ta...,"Climate change effects, global action, sustain...",The main points were the damage from climate...,"Climate damage, global cooperation needed, ren..."
2,TEXT,Explain your approach.,[Text],My approach was to carefully consider the var...,Systematic problem-solving,Logical breakdown and systematic solution.,Logical approach,"I just figured it out, you know? Took it ste...",Step-by-step solution,"I used a structured approach, beginning with...",Structured problem solving,I went about it in the most obvious way poss...,Straightforward approach
3,TEXT,Summarize the main points.,[Text],The main points were the rising cost of livin...,Economic crisis & solutions,"Rising costs, struggling businesses, governme...",Economic hardship,"Okay, so like, main points? Inflation's kill...",Inflation & business struggles,The primary focus was on the current economi...,Economic challenges & policy,The main points? Seriously? It was about i...,Economic problems & weak solutions
4,TEXT,What challenges did you encounter?,[Text],The biggest hurdle was definitely integrating...,Database integration problems,Time constraints and unexpected technical dif...,Time and tech issues,"Man, getting the data to play nice was a nig...",Data and client changes,"We faced several challenges, primarily relat...",Resource & scope issues,The constant shifting deadlines and lack of ...,Client communication & deadlines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,TEXT,Describe any areas where the convention fell s...,[Text],The convention's biggest letdown was the lack...,Poor Networking,Poorly organized sessions and insufficient b...,Poor Organization,"Dude, the wifi was AWFUL. And the food? Don...",Bad Wifi & Food,The promised keynote speakers failed to mater...,Speaker cancellations,The promised workshops were severely underwh...,Poor Workshops
174,TEXT,What aspects of the convention fostered a sens...,[Text],"The convention's commitment to accessibility,...",Accessibility & diverse programming,"Diverse programming, accessible venue, and w...",Key features,"They had stuff for everyone, ya know? Lots ...",Inclusivity & accessibility,The inclusive nature of the convention stemme...,Deliberate inclusivity design,"Well, it wasn't perfect, but they *did* try ...",Partial inclusivity efforts
175,TEXT,How effectively did the convention balance com...,[Text],The convention demonstrated a moderate level ...,"Moderate success, some compromises, lingering ...","Moderately successful, some compromises, but ...","Moderate success, imperfect compromise.","Yeah, it was alright, I guess. They kinda m...","Mixed results, some compromises, other failures.",The convention's effectiveness in balancing c...,"Varying success, compromise in some areas, fai...","It was a mixed bag, really. They made some p...","Mixed results, progress on some, neglect on ot..."
176,TEXT,What were the strengths and weaknesses of the ...,[Text],The convention's communication strategy effec...,"Social 

In [37]:
import pandas as pd

# Convert options to strings before melting
df_TEXT_with_answers['options'] = df_TEXT_with_answers['options'].astype(str)

# Step 1: Melt the dataframe to combine all answer columns into one column
df_long = df_TEXT_with_answers.melt(
    id_vars=["type", "question", "options"],
    var_name="answer_type",
    value_name="answer_value"
)


# Step 2: Create a new column for the answer type (e.g., verbose, concise, etc.)
df_long["answer_category"] = df_long["answer_type"].str.extract(r"answer_(\w+)")

# Step 3: Separate rows into answers and their labels
answers = df_long[~df_long["answer_type"].str.endswith("_Gemini_label")].copy()
labels = df_long[df_long["answer_type"].str.endswith("_Gemini_label")].copy()

# Step 4: Add a matching key column to facilitate merging
answers["label_type"] = answers["answer_type"] + "_Gemini_label"

# Step 5: Merge answers with their corresponding labels
df_reshaped = pd.merge(
    answers,
    labels,
    left_on=["type", "question", "options", "label_type"],
    right_on=["type", "question", "options", "answer_type"],
    suffixes=("", "_label")
)

# Step 6: Rename columns and keep the desired ones
df_reshaped = df_reshaped.rename(
    columns={"answer_value": "answer", "answer_value_label": "answer_label_GEMINI"}
)[["type", "question", "options", "answer", "answer_label_GEMINI", "answer_category"]]

# Step 7: Sort the data if needed (optional)
df_reshaped_TEXT = df_reshaped.sort_values(by=["type", "question"]).reset_index(drop=True)

# Display the final dataframe
df_reshaped_TEXT.to_csv('TEXT_answer_combined_total_reshaped.csv', index=False, sep='>')
df_reshaped_TEXT

,type,question,options,answer,answer_label_GEMINI,answer_category
0,TEXT,Describe any areas where cost savings could be...,['Text'],Significant cost savings could be realized by...,"Supplier negotiation, energy efficiency, proce...",verbose
1,TEXT,Describe any areas where cost savings could be...,['Text'],"Reduce supplier costs, improve energy efficie...",Cost reduction strategies,concise
2,TEXT,Describe any areas where cost savings could be...,['Text'],"Dude, we could totally save a ton of cash by...","Supplier negotiation, energy saving, waste red...",colloquial
3,TEXT,Describe any areas where cost savings could be...,['Text'],Cost savings can be achieved through a multi-...,"Contract review, energy efficiency, workflow o...",explanatory
4,TEXT,Describe any areas where cost savings could be...,['Text'],"Well, obviously we need to look at our spend...","Supplier negotiation, energy cost reduction",annoyed
...,...,...,...,...,...,...
895,TEXT,Would you recommend our support to others? Why...,['Text'],"Yes, I would definitely recommend your suppor...","Positive recommendation, detailed explanation",verbose
896,TEXT,Would you recommend our support to others? Why...,['Text'],Absolutely.,Positive recommendation,concise
897,TEXT,Would you recommend our support to others? Why...,['Text'],"Yeah, for sure! They totally sorted me out.",Casual positive recommendation,colloquial
898,TEXT,Would you recommend our support to others? Why...,['Text'],I would recommend your support based on my po...,"Positive recommendation, process explanation",explanatory


In [44]:
import glob

# list all csv files only
csv_files = ['TEXT_answer_combined_total_reshaped.csv', 'DATE_answer_combined_total_reshaped.csv', 'NUMBER_answer_combined_total_reshaped.csv', 'SINGLE_answer_combined_total_reshaped.csv', 'MULTI_answer_combined_total_reshaped.csv']

import pandas as pd

# Create an empty list to store each DataFrame
dfs = []

# Read each CSV file and append to the list
for file in csv_files:
    df = pd.read_csv(file, sep='>')
    dfs.append(df)

# Concatenate all DataFrames
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV
merged_df.to_csv('all_answers_combined_reshaped.csv', index=False, sep='>')


In [46]:
df_all = pd.read_csv('all_answers_combined_reshaped.csv', sep='>')
df_all

,type,question,options,answer,answer_label_GEMINI,answer_category
0,TEXT,Describe any areas where cost savings could be...,['Text'],Significant cost savings could be realized by...,"Supplier negotiation, energy efficiency, proce...",verbose
1,TEXT,Describe any areas where cost savings could be...,['Text'],"Reduce supplier costs, improve energy efficie...",Cost reduction strategies,concise
2,TEXT,Describe any areas where cost savings could be...,['Text'],"Dude, we could totally save a ton of cash by...","Supplier negotiation, energy saving, waste red...",colloquial
3,TEXT,Describe any areas where cost savings could be...,['Text'],Cost savings can be achieved through a multi-...,"Contract review, energy efficiency, workflow o...",explanatory
4,TEXT,Describe any areas where cost savings could be...,['Text'],"Well, obviously we need to look at our spend...","Supplier negotiation, energy cost reduction",annoyed
...,...,...,...,...,...,...
4485,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,We primarily utilize Facebook and Instagram fo...,"Facebook, Instagram, LinkedIn",verbose
4486,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,"Facebook, Instagram, and TikTok.","Facebook, Instagram, TikTok",concise
4487,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,"We're all over Facebook and Insta, mostly. Yo...","Facebook, Instagram, YouTube",colloquial
4488,MULTI_SELECT,Which social media platforms do you use for ma...,Facebook; Instagram; Twitter; LinkedIn; TikTok...,Our marketing strategy uses a multi-platform a...,"Facebook, Instagram, YouTube, LinkedIn",explanatory


In [48]:
df_all['options'] = df_all['options'].str.replace(r"\['|'\]", "", regex=True)

In [50]:
df_all.to_csv('all_answers_combined_reshaped.csv', index=False, sep='>')

# Notes

In [ ]:
'''
import pandas as pd
from transformers import pipeline

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Example DataFrame
data = {
    "text": [
        "I prefer to eat mostly bread.",
        "I love grilled salmon.",
        "Steak is my favorite dish.",
        "I enjoy cheeseburgers."
    ],
    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row
               ["Fish", "Meat", "Bread", "Burger"],
               ["Meat", "Burger", "Fish", "Bread"],
               ["Burger", "Meat", "Fish", "Bread"]]
}

df_test = pd.DataFrame(data)

df_test


# Function to classify each row
def classify_row(row):
    result = classifier(row["text"], candidate_labels=row["labels"])
    return result["labels"][0], result["scores"][0]


# Apply the classification to each row

df["predicted_labels"], df["scores"] = zip(*df.apply(lambda row: classify_row(row), axis=1))

# Display the DataFrame
print(df)
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


'\n# Function to classify each row\ndef classify_row(row):\n    result = classifier(row["text"], candidate_labels=row["labels"])\n    return result["labels"][0], result["scores"][0]\n\n\n# Apply the classification to each row\n\ndf["predicted_labels"], df["scores"] = zip(*df.apply(lambda row: classify_row(row), axis=1))\n\n# Display the DataFrame\nprint(df)\n'

In [ ]:
'''
import pandas as pd

data_test = {
    "text": [
        "I prefer to eat mostly bread.",
        "I love grilled salmon.",
        "Steak is my favorite dish.",
        "I enjoy cheeseburgers."
    ],
    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row
               ["Fish", "Meat", "Bread", "Burger"],
               ["Meat", "Burger", "Fish", "Bread"],
               ["Burger", "Meat", "Fish", "Bread"]]
}

df_test = pd.DataFrame(data)

df_test

print(df)
'''

'\nimport pandas as pd \n\ndata_test = {\n    "text": [\n        "I prefer to eat mostly bread.",\n        "I love grilled salmon.",\n        "Steak is my favorite dish.",\n        "I enjoy cheeseburgers."\n    ],\n    "labels": [["Bread", "Fish", "Meat", "Burger"],  # Labels for each row\n               ["Fish", "Meat", "Bread", "Burger"],\n               ["Meat", "Burger", "Fish", "Bread"],\n               ["Burger", "Meat", "Fish", "Bread"]]\n}\n\ndf_test = pd.DataFrame(data)\n\ndf_test\n\nprint(df)\n'